In [60]:
import pandas as pd
#import yfinance as yf
from yahoofinancials import YahooFinancials as yfs
import json

# !pip install yahoofinancials

In [61]:
def Sheet_Data(ticker,sheet_data):
    save = []
    
    df = pd.DataFrame.from_dict({(i,j): sheet_data[i][j] 
                            for i in sheet_data.keys() 
                            for j in sheet_data[i].keys()},
                        orient='index')

    for i in df.columns:
        f = len(df.columns) - i - 1
        df2 = pd.json_normalize(df[f],max_level = 1)

        #Gets  Date and renames 
        Date = df2.columns[0]
        Date = Date[0:11]
        df2.index = df2.index.rename(ticker)
        df2.columns = df2.columns.str.replace(str(Date),'')

        #Tranposes DataFrame and changes it to the 
        df2 = df2.T
        df2.rename(columns = {df2.columns[0]:'{}'.format(str(Date[0:4]))},inplace = True)
        #display(df2)
        save.append(df2)
    df3 = pd.concat(save,axis = 1)
    #display(df3)
    filename = str(df.index[0][1]) + '_' + str(df.index[0][0])
    print(filename)
    #df3.to_csv('{}.csv'.format(filename))
    return df3
#filename = str(df.index[0][1]) + '_' + str(df.index[0][0])
#df3.t

In [62]:
ticker = 'JPM'
yahoo_financials = yfs(ticker)

cash = yahoo_financials.get_financial_stmts('annual','cash')
income = yahoo_financials.get_financial_stmts('annual','income')
balance = yahoo_financials.get_financial_stmts('annual','balance')
stock = yahoo_financials.get_key_statistics_data()[ticker]

cash_balance = Sheet_Data(ticker,cash)
income_balance = Sheet_Data(ticker,income)
balance_balance = Sheet_Data(ticker,balance)

MU_cashflowStatementHistory
MU_incomeStatementHistory
MU_balanceSheetHistory


In [63]:
#WACC = Wd * Rd * (1 - t) + We + Re
INCOME2 = income_balance.loc[['incomeBeforeTax','incomeTaxExpense']]
t = income_balance.loc['incomeBeforeTax']/ income_balance.loc['incomeTaxExpense']
t = t[-1]/ 100 



yahoo_financials.get_beta()
df = pd.read_html('https://finance.yahoo.com/bonds')
Rf = float(df[0]['Last Price'][2])
try: 
    beta = float(yahoo_financials.get_beta())
except:
    beta = 0
Rm = float(10.0)
Ra = Rf + beta * (Rm - Rf)
#print(Ra)

try:
    Balance2 = balance_balance.loc[['shortLongTermDebt','longTermDebt']]#.astype#(float)
except:
    Balance2 = balance_balance.loc[['longTermDebt']]

Total_debt = Balance2.sum()[-1] 
Balance2 =   -(income_balance.loc['interestExpense'][-1]) / Total_debt
Rd = (1-Balance2) * t #  Rd * (1 - t)
#Balance2.sum()


Market_Cap = yahoo_financials .get_market_cap() # MarketCap
Total =  Total_debt + Market_Cap 
Wd = Total_debt/ Total 
We = Market_Cap/ Total 
Re = Ra
Required_Rate = (Wd * Rd + We * Re)/100
Required_Rate

FCF = cash_balance.loc[['totalCashFromOperatingActivities','capitalExpenditures','netIncome']] # how to select rows 
Revenue = income_balance.loc[['totalRevenue','netIncome']]
Growth_Rate = Revenue.pct_change().mean()
#FCF is CFO - CAPEX = FCF
#FCF = cash_balance.loc[['totalCashFromOperatingActivities','capitalExpenditures','netBorrowings']] # More 

df = pd.DataFrame()
df['FCF'] = FCF.loc['totalCashFromOperatingActivities'] + FCF.loc['capitalExpenditures']
df['Net Income'] = FCF.loc['netIncome']
#df['FCF/Net Income'] = df['FCF']/df['Net Income']
minimum = (df['FCF']/df['Net Income']).min()
print(Wd)
print(Rd)
print(We)
print(Re)

print(Ra)
print(Rf)

0.0590075056194681
0.10342101904916588
0.9409924943805319
12.215863405
12.215863405
1.635


# Anaylsis

In [64]:
def DCF_VALUE():
    outstanding_shares = stock['sharesOutstanding']

    Required_rate = Required_Rate
    Pre_rate = .025
    Cashflowgrowthrate = minimum

    years = [1,2,3,4]
    freecash_flow = df['FCF'].values

    terminalvalue = freecash_flow[-1] * (1+Pre_rate)/(Required_rate - Pre_rate)

    futurefreecashflow = []
    discountfactor = []
    discountfuturefreecashflow = []
    for  year in years:
        cashflow = freecash_flow[-1] * (1+ Cashflowgrowthrate) ** year 
        futurefreecashflow.append(cashflow)
        discountfactor.append((1+ Required_rate)**year)


    for i in range(0,len(years)):
        discountfuturefreecashflow.append(futurefreecashflow[i]/discountfactor[i])

    print(discountfuturefreecashflow)
    dis = terminalvalue/(1+Required_rate)**len(years)
    discountfuturefreecashflow.append(dis)
    value = sum(discountfuturefreecashflow)
    fairvalue = value/outstanding_shares

    print(ticker)
    print(fairvalue)
    print(yahoo_financials.get_current_price())

    print(Cashflowgrowthrate)
    print(Required_Rate)
#print(stock.get_current_price())
DCF_VALUE()

[76738073.76441212, 70948577.89243801, 65595869.9251813, 60646996.44529331]
MU
0.7914904976551931
87.77
0.030889467807964272
0.11501138392845522


In [65]:
def DCF_VALUE_CON():
    outstanding_shares = stock['sharesOutstanding']

    Required_rate = .075
    Pre_rate = .025
    Cashflowgrowthrate = .035

    years = [1,2,3,4]
    freecash_flow = df['FCF'].values

    terminalvalue = freecash_flow[-1] * (1+Pre_rate)/(Required_rate - Pre_rate)

    futurefreecashflow = []
    discountfactor = []
    discountfuturefreecashflow = []
    for  year in years:
        cashflow = freecash_flow[-1] * (1+ Cashflowgrowthrate) ** year 
        futurefreecashflow.append(cashflow)
        discountfactor.append((1+ Required_rate)**year)


    for i in range(0,len(years)):
        discountfuturefreecashflow.append(futurefreecashflow[i]/discountfactor[i])

    print(discountfuturefreecashflow)
    dis = terminalvalue/(1+Required_rate)**len(years)
    discountfuturefreecashflow.append(dis)
    value = sum(discountfuturefreecashflow)
    fairvalue = value/outstanding_shares

    print(ticker)
    print(fairvalue)
    print(yahoo_financials.get_current_price())

    print(Cashflowgrowthrate)
    print(Required_rate)
#print(stock.get_current_price())
DCF_VALUE_CON()

[79911627.90697674, 76938171.98485668, 74075356.28309457, 71319063.95628174]
MU
1.4091093791582523
87.77
0.035
0.075


df = pd.concat([df2[2],df2[3],df2[4]])
#display(df)
x = []
x.append(df['Unnamed: 0'].dropna())
x.append(df['Liabilities & Shareholders Equity'].dropna())
x.append(df['Shareholders Equity'].dropna())

y = pd.concat(x).tolist()
df = df.drop(['Liabilities & Shareholders Equity','Shareholders Equity','Unnamed: 0'], axis=1)
df['Liabilities & Shareholders Equity'] = y
df.set_index('Liabilities & Shareholders Equity')

x#Anaylist esitmates 
df = pd.read_html('https://finance.yahoo.com/quote/AAPL/analysis?p=AAPL')
df = df[1] 
df.index = df[df.columns[0]]
df1 = df.index 
df1 = df[df.columns[3:5]]
df1 = df1.drop(['Sales Growth (year/est)'])

for i in df1.columns:
    df1[i] = df1[i].str.replace('B','e9')
    df1[i] = df1[i].astype(float)

#df1.loc['Avg. Estimate']
df1.loc['Avg. Estimate'].tolist()

Revenue1 = Revenue.loc['totalRevenue']
strs = int(Revenue.columns[3])
Revenue1['{}'.format(strs+1)] = df1.loc['Avg. Estimate'][0]
Revenue1['{}'.format(strs+2)] = df1.loc['Avg. Estimate'][1]
growth = 1 + Revenue.T.pct_change().sum()/5

Revenue1['{}'.format(strs+3)] = l.drop_index() * growth
Revenue1['{}'.format(strs+4)] = l2.drop_index() * growth
Revenue.loc['netIncome']/Revenue.loc['totalRevenue']

ticker = 'LMT'
yahoo_financials = yfs(ticker)

cash = yahoo_financials.get_financial_stmts('annual','cash')
income = yahoo_financials.get_financial_stmts('annual','income')
balance = yahoo_financials.get_financial_stmts('annual','balance')
stock = yahoo_financials.get_key_statistics_data()[ticker]

sample = pd.DataFrame()
sample['Free Cash Flow'] = df['FCF']
sample['Cash Flow'] = futurefreecashflow
sample['discountfactor'] = discountfactor
sample

df = pd.DataFrame.from_dict({(i): stock[i] 
                            for i in stock.keys() 
                            },
                        orient='index')
df

aapl.info['sharesOutstanding']
stock.keys()

pd.DataFrame.from_dict({(i): stock[i] 
for i in stock.keys()},orient='index')

# Testing for working all of the files work

df = pd.DataFrame.from_dict({(i,j): balance_sheet_data[i][j] 
                           for i in balance_sheet_data.keys() 
                           for j in balance_sheet_data[i].keys()},
                       orient='index')
for i in df.columns:
    df[i].tolist()
    df2 = pd.json_normalize(df[i],max_level = 1)

    #Gets  Date and renames 
    Date = df2.columns[0]
    Date = Date[0:11]
    df2.index = df2.index.rename(ticker)
    df2.columns = df2.columns.str.replace(str(Date),'')

    #Tranposes DataFrame and changes it to the 
    df2 = df2.T
    df2.rename(columns = {df2.columns[0]:'{}'.format(str(Date[0:10]))},inplace = True)
    #display(df2)
    save.append(df2)
df3 = pd.concat(save,axis = 1)
display(df3)